## 1) Execution Summary

This report documents a churn-prediction BI workflow implemented in Streamlit and reproduced here for reproducibility.

**Scope**

- Dataset A: **Telco churn** (`telco_customer_churn.csv`) — target column: `Churn`.
- Dataset B: **Bank churn** (`churn.csv`) — target column: `Exited`.

**What was done**

- Loaded and profiled both datasets (schema, missingness, class balance).
- Applied preprocessing + feature engineering consistent with the Streamlit app.
- Trained and compared four classification models on each dataset: SVM, Logistic Regression, Random Forest, Gradient Boosting.
- Reported performance using accuracy, precision, recall, F1, confusion matrices, and ROC-AUC.


In [ ]:
# Imports
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from pandas.api.types import is_numeric_dtype

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
    roc_curve,
    auc,
)

sns.set_style('whitegrid')

In [ ]:
# Load datasets
telco_path = 'telco_customer_churn.csv'
bank_path = 'churn.csv'

telco_raw = pd.read_csv(telco_path)
bank_raw = pd.read_csv(bank_path)

telco_raw.shape, bank_raw.shape

## 2) Comparison of Datasets

This section compares: size, schema, feature types, missing values, duplicates, and target distributions.


In [ ]:
def dataset_profile(df: pd.DataFrame, target: str):
    out = {}
    out['rows'] = df.shape[0]
    out['cols'] = df.shape[1]
    out['object_cols'] = df.select_dtypes(include=['object']).columns.tolist()
    out['numeric_cols'] = df.select_dtypes(include=[np.number]).columns.tolist()
    out['missing_total'] = int(df.isna().sum().sum())
    out['duplicate_rows'] = int(df.duplicated().sum())
    if target in df.columns:
        out['target_value_counts'] = df[target].value_counts(dropna=False).to_dict()
    else:
        out['target_value_counts'] = None
    return out

telco_profile = dataset_profile(telco_raw, target='Churn')
bank_profile = dataset_profile(bank_raw, target='Exited')
pd.DataFrame([
    {'dataset':'Telco', **telco_profile},
    {'dataset':'Bank', **bank_profile},
]).set_index('dataset')

In [ ]:
# Missing values per dataset (top 15)
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

telco_missing = telco_raw.isna().sum().sort_values(ascending=False).head(15)
bank_missing = bank_raw.isna().sum().sort_values(ascending=False).head(15)

axes[0].bar(telco_missing.index.astype(str), telco_missing.values)
axes[0].set_title('Telco: Missing Values (Top 15)')
axes[0].tick_params(axis='x', rotation=90)

axes[1].bar(bank_missing.index.astype(str), bank_missing.values)
axes[1].set_title('Bank: Missing Values (Top 15)')
axes[1].tick_params(axis='x', rotation=90)

plt.tight_layout()
plt.show()

### Key Observations (fill after running)

- **Telco**: _(describe class balance, missing TotalCharges handling, categorical richness, etc.)_
- **Bank**: _(describe Geography/Gender categories, numeric-heavy schema, target imbalance, etc.)_


## 3) Methodology and Choices

### Modeling strategy

- Split: train/test using `train_test_split`.
- Models compared: SVM, Logistic Regression (scaled), Random Forest, Gradient Boosting.
- Metrics: accuracy, precision, recall, F1, ROC-AUC + confusion matrix.

### Preprocessing choices

- Telco: numeric coercion for `TotalCharges`, binary encoding for Yes/No and gender, one-hot for multi-class columns.
- Bank: drop IDs, encode Gender, one-hot Geography, feature engineering (age groups, tenure groups, credit risk, balance risk, etc.).


In [ ]:
# Telco preprocessing (aligned with streamlit.py)
def preprocess_telco(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Coerce TotalCharges -> numeric and impute median
    if 'TotalCharges' in df.columns:
        df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
        df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median())

    # Consolidate 'No internet service' -> 'No'
    for col in df.columns:
        try:
            uniques = df[col].dropna().unique().tolist()
        except Exception:
            continue
        if set(uniques) == set(['No internet service', 'No', 'Yes']):
            df[col] = df[col].replace({'No internet service': 'No'})

    # Consolidate 'No phone service' -> 'No'
    for col in df.columns:
        try:
            uniques = df[col].dropna().unique().tolist()
        except Exception:
            continue
        if set(uniques) == set(['No phone service', 'No', 'Yes']):
            df[col] = df[col].replace({'No phone service': 'No'})

    # Binary mappings
    for col in df.columns:
        try:
            uniques = sorted(pd.Series(df[col].dropna().unique()).astype(str).tolist())
        except Exception:
            continue
        if uniques == ['No', 'Yes']:
            df[col] = df[col].map({'No': 0, 'Yes': 1})
        if uniques == ['Female', 'Male']:
            df[col] = df[col].map({'Male': 0, 'Female': 1})

    # Ensure required multiclass columns exist
    for needed in ['InternetService', 'Contract', 'PaymentMethod']:
        if needed not in df.columns:
            df[needed] = 'Unknown'

    dummies = pd.get_dummies(
        df,
        dtype=int,
        columns=['InternetService', 'Contract', 'PaymentMethod'],
        drop_first=True,
    )

    if 'Churn' in dummies.columns and not is_numeric_dtype(dummies['Churn']):
        if dummies['Churn'].dtype == object:
            dummies['Churn'] = dummies['Churn'].map({'No': 0, 'Yes': 1})

    return dummies

telco = preprocess_telco(telco_raw)
telco.head()

In [ ]:
# Bank preprocessing + feature engineering (aligned with your notebook + streamlit.py additions)
def preprocess_bank(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Drop identifiers
    df = df.drop(columns=[c for c in ['RowNumber', 'CustomerId', 'Surname'] if c in df.columns])

    # Fill missing values
    for col in df.columns:
        if col == 'Exited':
            continue
        if is_numeric_dtype(df[col]):
            df[col] = df[col].fillna(df[col].median())
        else:
            df[col] = df[col].fillna('Unknown')

    # Gender
    if 'Gender' in df.columns:
        df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0}).fillna(0).astype(int)

    # Geography one-hot
    if 'Geography' in df.columns:
        geo_dummies = pd.get_dummies(df['Geography'], drop_first=True, dtype=int)
        df = pd.concat([df.drop(columns=['Geography']), geo_dummies], axis=1)

    # Feature engineering
    if 'Balance' in df.columns and 'EstimatedSalary' in df.columns:
        df['Balance_to_Salary_Ratio'] = df['Balance'] / (df['EstimatedSalary'] + 1)
        df['Is_Wealthy'] = (df['Balance'] > df['Balance'].quantile(0.75)).astype(int)

    if 'Age' in df.columns:
        age_group = pd.cut(df['Age'], bins=[0, 30, 40, 50, 60, 100], labels=['18-30', '31-40', '41-50', '51-60', '60+'])
        df['Age_Group'] = pd.Categorical(age_group).codes
        df['Is_Senior'] = (df['Age'] > 60).astype(int)

    if 'Tenure' in df.columns:
        tenure_group = pd.cut(df['Tenure'], bins=[0, 1, 3, 5, 10, float('inf')], labels=['New', 'Developing', 'Mature', 'Loyal', 'Very_Loyal'], include_lowest=True)
        df['Tenure_Group'] = pd.Categorical(tenure_group).codes
        df['Is_New_Customer'] = (df['Tenure'] <= 1).astype(int)

    if 'CreditScore' in df.columns:
        credit_risk = pd.cut(df['CreditScore'], bins=[0, 400, 600, 750, 850, 1000], labels=['Very_High', 'High', 'Medium', 'Low', 'Very_Low'])
        df['Credit_Risk'] = pd.Categorical(credit_risk).codes

    if 'NumOfProducts' in df.columns:
        df['Num_Products'] = df['NumOfProducts']
        df['Has_Multiple_Products'] = (df['Num_Products'] > 1).astype(int)

    if 'IsActiveMember' in df.columns and 'Tenure' in df.columns:
        df['Activity_Tenure_Score'] = df['IsActiveMember'] * df['Tenure']

    if 'Age' in df.columns and 'Num_Products' in df.columns:
        df['Age_Product_Interaction'] = df['Age'] * df['Num_Products']

    if 'Balance' in df.columns:
        df['Zero_Balance'] = (df['Balance'] == 0).astype(int)
        df['Low_Balance'] = (df['Balance'] < df['Balance'].quantile(0.25)).astype(int)

    if 'Exited' in df.columns:
        df['Exited'] = pd.to_numeric(df['Exited'], errors='coerce').fillna(0).astype(int)

    return df

bank = preprocess_bank(bank_raw)
bank.head()

## 4) Results and Model Performance

The next cells train a consistent set of models per dataset and summarize performance in a table.


In [ ]:
def evaluate_models(X: pd.DataFrame, y: pd.Series, random_state: int = 42, test_size: float = 0.2, title_prefix: str = ''):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y if y.nunique() == 2 else None)

    models = {}
    # SVM (scaled)
    models['SVM (RBF)'] = Pipeline([('scaler', StandardScaler()), ('svm', SVC(kernel='rbf', probability=True, random_state=random_state))])

    # Logistic Regression (scaled + simple CV over max_iter, matching your Streamlit pattern)
    lr_pipe = Pipeline([('scaler', StandardScaler()), ('logreg', LogisticRegression())])
    param_grid = {'logreg__max_iter': [500, 600, 700, 800, 900, 1000]}
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    models['LogReg (CV)'] = GridSearchCV(lr_pipe, param_grid, cv=kf)

    # Random Forest
    models['RandomForest'] = RandomForestClassifier(random_state=random_state, n_estimators=200)

    # Gradient Boosting
    models['GradientBoosting'] = GradientBoostingClassifier(random_state=random_state)

    rows = []
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # proba for ROC-AUC when available
        roc_auc = np.nan
        if hasattr(model, 'predict_proba'):
            y_proba = model.predict_proba(X_test)[:, 1]
            fpr, tpr, _ = roc_curve(y_test, y_proba)
            roc_auc = auc(fpr, tpr)

        rows.append({
            'model': name,
            'accuracy': float(model.score(X_test, y_test)),
            'precision': float(precision_score(y_test, y_pred, zero_division=0)),
            'recall': float(recall_score(y_test, y_pred, zero_division=0)),
            'f1': float(f1_score(y_test, y_pred, zero_division=0)),
            'roc_auc': float(roc_auc) if roc_auc == roc_auc else np.nan,
        })

        # Confusion matrix plot
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(4.5, 3.5))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title(f'{title_prefix}{name} — Confusion Matrix')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.tight_layout()
        plt.show()

    return pd.DataFrame(rows).sort_values(by=['f1', 'roc_auc'], ascending=False).reset_index(drop=True)

In [ ]:
# Telco training
telco_target = 'Churn'
X_telco = telco.drop(columns=[telco_target])
y_telco = telco[telco_target]

telco_results = evaluate_models(X_telco, y_telco, title_prefix='Telco: ')
telco_results

In [ ]:
# Bank training
bank_target = 'Exited'
X_bank = bank.drop(columns=[bank_target])
y_bank = bank[bank_target]

bank_results = evaluate_models(X_bank, y_bank, title_prefix='Bank: ')
bank_results

### Interpretation (fill after running)

- Best Telco model: _(fill from `telco_results`)_
- Best Bank model: _(fill from `bank_results`)_

Discuss trade-offs: interpretability (LogReg) vs performance (tree ensembles), and deployment considerations (latency, feature stability).


## 5) Screenshots of the Streamlit Interface

Add 2–6 screenshots (recommended):

1. Dataset tabs (Telco vs Bank)
2. Telco model training results
3. Telco prediction UI
4. Bank model training results
5. Bank prediction UI

### How to capture

- Run: `streamlit run streamlit.py`
- Take screenshots and save them under `./screenshots/` (create the folder).

### Embed in this report

Replace the placeholder paths below once images exist.

![Streamlit — Dataset Tabs](screenshots/01_tabs.png)

![Streamlit — Telco Training](screenshots/02_telco_training.png)

![Streamlit — Telco Prediction](screenshots/03_telco_prediction.png)

![Streamlit — Bank Training](screenshots/04_bank_training.png)

![Streamlit — Bank Prediction](screenshots/05_bank_prediction.png)


## 6) Conclusions and Recommendations

**Conclusions (template)**

- The two datasets differ significantly in feature composition: Telco is category-heavy with service attributes; Bank is numeric-heavy with customer/finance features.
- Model performance is dataset-dependent; tree ensembles often perform strongly, while Logistic Regression provides interpretability.

**Recommendations**

- Monitor data drift: categorical distributions (Telco) and numeric ranges (Bank) can shift over time.
- Add calibration / threshold tuning if the business cost of false positives/negatives is asymmetric.
- Consider explaining predictions (feature importance / SHAP) before production rollout.
- Validate fairness and bias across sensitive attributes where applicable (e.g., gender).


## 7) Deployment Link (if applicable)

- Streamlit deployment link: **_(paste your URL here)_**

If not deployed yet, options include Streamlit Community Cloud, Hugging Face Spaces, or a small VM/container.
